In [7]:
import cv2 
import datetime
import numpy as np

keys = {"+":False,"-":False,"s":False,"a":False,"d":False,"f":False,"b":False,"g":False,"c":False,"q":False,"r":False}

drawn = False

x1,x2,y1,y2 = 0,0,0,0

zoom_factor = 1.0

rotation_angle = 0.0

sharpen = False

gray = False

canny = False

blur = False

current_flip_direction = 0

sharpening_kernel = np.array([[-1,-1,-1],
                               [-1,9,-1],
                               [-1,-1,-1]])

def pressed_not(key):
    if key in keys:
        keys[key] = not keys[key]

def zoom_roi(frame,factor):
    global x1,x2,y1,y2
    w = (x2-1) - (x1+1)
    h = (y2-1) - (y1+1)
    center_x = (x1+x2)//2
    center_y = (y1+y2)//2
    new_w = int(w/factor)
    new_h = int(h/factor)
    new_x1 = max(center_x - new_w // 2,0)
    new_y1 = max(center_y - new_h // 2,0)
    new_x2 = new_x1 + new_w
    new_y2 = new_y1 + new_h
    roi_region = frame[new_y1:new_y2,new_x1:new_x2]
    zoomed_roi = cv2.resize(roi_region,(w,h))
    frame[y1+1:y2-1,x1+1:x2-1] = zoomed_roi
    return frame

def flip_roi(roi):
    global current_flip_direction
    if current_flip_direction == 0:
        flipped_frame = roi
    elif current_flip_direction == 1:
        flipped_frame = cv2.flip(roi,1)
    elif current_flip_direction == 2:
        flipped_frame = cv2.flip(roi,0)
    elif current_flip_direction == 3:
        flipped_frame = cv2.flip(roi,-1)
    return flipped_frame

def rotate_roi(roi,angle):
    center = (roi.shape[1]//2,roi.shape[0]//2)
    rotation_matrix = cv2.getRotationMatrix2D(center,angle,1.0)
    rotated_roi = cv2.warpAffine(roi,rotation_matrix,(roi.shape[1],roi.shape[0]))
    return rotated_roi

def sharpen_roi(roi):
    sharpened_roi = cv2.filter2D(roi,-1,sharpening_kernel)
    return sharpened_roi

def call_back(event,x,y,flags,params):
    global x1,y1,x2,y2,drawn

    if event == cv2.EVENT_LBUTTONDOWN:
        x1,y1 = max(0,min(frame.shape[1],x)) , max(0,min(frame.shape[0],y))
        drawn = False
    
    elif event == cv2.EVENT_LBUTTONUP:
        x2,y2 = max(0,min(frame.shape[1],x)) , max(0,min(frame.shape[0],y))

        if x2<x1:
            x1,x2 = x2,x1
        
        if y2<y1:
            y1,y2 = y2,y1

        drawn = True

cv2.namedWindow("Camera")
cv2.setMouseCallback("Camera",call_back)

cap = cv2.VideoCapture(0)

while cap.isOpened():
    state,frame = cap.read()

    if state:
        frame = cv2.flip(frame,1)

        if drawn:
            cv2.rectangle(frame,(x1,y1),(x2,y2),(255,255,255),1)

            roi = frame[y1+1:y2-1,x1+1:x2-1]

            if zoom_factor != 1.0:
                frame = zoom_roi(frame,zoom_factor)

            if rotation_angle != 0:
                rotated_roi = rotate_roi(roi,rotation_angle)
                frame[y1+1:y2-1,x1+1:x2-1] = rotated_roi

            if sharpen:
                frame[y1+1:y2-1,x1+1:x2-1] = sharpen_roi(roi)

            if current_flip_direction != 0:
                frame[y1+1:y2-1,x1+1:x2-1] = flip_roi(roi)

            if gray:
                gray_roi = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
                if canny:
                    gray_roi = cv2.Canny(roi,50,150)
                frame[y1+1:y2-1,x1+1:x2-1] = cv2.cvtColor(gray_roi,cv2.COLOR_GRAY2BGR)

            if blur:
                frame[y1+1:y2-1,x1+1:x2-1] = cv2.GaussianBlur(roi,(5,5),0)
            
            if rotation_angle == 0 or rotation_angle%360==0:
                keys["a"] = False
                keys["d"] = False
        
        frame_whole = frame.copy()

        text = "Pressed Keys: "
        for key in keys:
            if keys[key]:
                text+= f"{key} "
        
        cv2.putText(frame,text,(30,50),cv2.FONT_ITALIC,.6,(0,0,255),2)

        cv2.imshow("Camera",frame)

        k = cv2.waitKey(1)
        
        if k==ord("q"):
            break

        if x1!=x2 and y2!=y1:
            if k==ord("+"):
                zoom_factor = min(zoom_factor+0.2,4.0)
                keys["+"] = (zoom_factor != 1.0)
            if k==ord("-"):
                zoom_factor = max(zoom_factor-0.2,1.0)
                keys["-"] = (zoom_factor != 1.0)
                keys["+"] = (zoom_factor != 1.0)
            if k==ord("a"):
                rotation_angle -= 5.0
                keys["a"] = (rotation_angle%360!=0)
            if k==ord("d"):
                rotation_angle += 5.0
                keys["d"] = (rotation_angle%360!=0)
            if k==ord("f"):
                current_flip_direction = (current_flip_direction+1)%4
                keys["f"] = current_flip_direction!=0
            if k==ord("s"):
                pressed_not("s")
                sharpen = not sharpen
            if k==ord("g"):
                pressed_not("g")
                gray = not gray
            if k==ord("c"):
                pressed_not("c")
                canny = not canny
            if k==ord("b"):
                pressed_not("b")
                blur = not blur
            if k==ord("r"):
                gray = False
                canny = False
                blur = False
                sharpen = False
                zoom_factor = 1.0
                rotation_angle = 0.0
                current_flip_direction = 0
                keys = {"+":False,"-":False,"s":False,"a":False,"d":False,"f":False,"b":False,"g":False,"c":False,"q":False,"r":False}
            if k==ord("o"):
                timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
                cv2.imwrite(f"{timestamp}.png",frame_whole)
            if k==ord("m"):
                timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
                cv2.imwrite(f"{timestamp}.png",frame_whole[y1+1:y2-1,x1+1:x2-1])
    else:
        break
cap.release()
cv2.destroyAllWindows()
